In [3]:
import sys
if r"D:\Data\Google Drive\Openclassrooms\P7\Projet\src" not in sys.path:
    sys.path.append(r"D:\Data\Google Drive\Openclassrooms\P7\Projet\src")

import importlib

import config

import pandas as pd
import numpy as np

import authenticate
import api_sentiment_analysis
import func

import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

importlib.reload(config)
importlib.reload(authenticate)
importlib.reload(api_sentiment_analysis)
importlib.reload(func)

from authenticate import authenticate_client
from api_sentiment_analysis import api_sentiment_analysis

key = config.COGNITIVE_SERVICE_KEY
endpoint = config.SENTIMENT_ANALYSIS_ENDPOINT

data_path = "D:\Data\Google Drive\Openclassrooms\P7\Projet\data\data.csv"


In [4]:
# Read and sample the tweets
df = pd.read_csv(data_path,  encoding="ISO-8859-1", usecols=[0, 5], names=["label","tweet"])
tweets, true_label = func.sample_tweets(df, 2000)

In [3]:
# # Instantiate the client
# client = authenticate_client(key, endpoint)

# # Call the sentiment analysis API
# # Batches of 9 tweets per call
# results = []
# n_batches = int(np.ceil(len(tweets)/10))
# for i in range(n_batches):
#     result = api_sentiment_analysis(client, tweets[i*10:(i+1)*10])
#     results.extend(result)
#     pass

# print(f"Length of results: {len(results)}")

Length of results: 2000


In [4]:
# # Save api predictions
# with open(r".\api-results\results.p", 'wb') as f:
#     pickle.dump(results, f)
#     pass

In [5]:
# Load api predictions
with open(r".\api-results\results.p", 'rb') as f:
    results = pickle.load(f)

print(f"Length of results: {len(results)}")
print(len(results)==len(tweets))

Length of results: 2000
True


In [12]:
X = [(results[i]['confidence_scores']['positive'],
    results[i]['confidence_scores']['neutral'],
    results[i]['confidence_scores']['negative']) 
    for i in range(len(results))]

print(f"Length of results: {len(X)}")
print(len(results)==len(tweets))

Length of results: 2000
True


In [56]:
# Décision simple sur la sortie de l'API
trans = {'0': 1, '1':-1 , '2':0}

##############################################################
print("Score hors tweets prédits comme neutres")

predict = []
for i in range(len(X)):
    argmax= np.argmax(X[i])
    predict.append(trans[str(argmax)])
predict = np.array(predict)

# Pourcentage de tweets labellisés 'neutre'
print(f"% de score neutre: {(predict==-1).sum() / len(predict) }")

# Score sur les tweets hors ceux labellisés neutres
idx = (predict!=-1)
pred = predict[idx]
true_label_ = np.array(true_label)[idx]
print("Score sur les tweets hors neutres")
print("Accuracy score: ",accuracy_score(true_label_, pred))
print("F1 score: ",f1_score(true_label_, pred))
print("AUC score: ",roc_auc_score(true_label_, pred))

##############################################################
print("\nScore global")

predict = []
for i in range(len(X)):
    argmax= np.argmax((X[i][0], 0, X[i][2]))
    predict.append(trans[str(argmax)])
predict = np.array(predict)

# Pourcentage de tweets labellisés 'neutre'
print(f"% de prédiction neutre: {(predict==-1).sum() / len(predict) }")

# Score
idx = (predict!=-1)
pred = predict[idx]
true_label_ = np.array(true_label)[idx]
print("Score sur les tweets hors neutres")
print("Accuracy score: ",accuracy_score(true_label_, pred))
print("F1 score: ",f1_score(true_label_, pred))
print("AUC score: ",roc_auc_score(true_label_, pred))

Score hors tweets prédits comme neutres
% de score neutre: 0.2575
Score sur les tweets hors neutres
Accuracy score:  0.769023569023569
F1 score:  0.7677725118483412
AUC score:  0.7704365988264292

Score global
% de prédiction neutre: 0.0
Score sur les tweets hors neutres
Accuracy score:  0.731
F1 score:  0.7485981308411215
AUC score:  0.7310000000000001
